In [1]:
# unset PYTHONPATH first
from ko_text import *
from ko_crawler import *

In [2]:
nlp = NLP()

In [4]:
train_df = pd.read_csv('Data/Train_final.csv', encoding = 'utf-8')
test_df = pd.read_csv('Data/Test_final.csv', encoding = 'utf-8')

# 용량을 줄이기 위해 '단어 단어' 꼴로 묶어둔 token을 ['단어', '단어'] 꼴로 풀기
train_df['Token'] = [token.split() for token in train_df['Token']]
test_df['Token'] = [token.split() for token in test_df['Token']]

In [5]:
print(train_df.shape)

(87899, 2)


In [6]:
Counter(train_df['Section'])

Counter({'IT과학': 9996,
         '경제': 9660,
         '국제': 9844,
         '기업': 9965,
         '문화': 9846,
         '부동산': 9986,
         '사회': 9880,
         '정치': 9295,
         '증권': 9427})

In [7]:
train_df.head()

,Token,Section
0,"[NH, 농협, 금융, 지주, 김광수, 회장, 22일, 서대문, 본사, 에서, 농협...",경제
1,"[바이오, 직스, 올해, 개별, 기준, 3분, 영업, 이익, 105억원, 으로, 작...",경제
2,"[코스피, 붕괴하면서, 달러, 환율, 으로, 뛰었다, 23일, 서울, 외환시장, 에...",경제
3,"[정의당, 국회, 롯데, 갑질, 피해자, 간담, 개최, 김상조, 정거, 위원장, 2...",경제
4,"[코스피, 23일, 무역, 전쟁, 내외, 악재, 힘없이, 2110, 내줬다, 코스피...",경제


In [8]:
# 한 label마다 학습할 단어의 수
train_size_for_each_label = 1000
test_size_for_each_label = 300

n_class = len(train_df['Section'].unique())


# Doc2Vec으로 vector를 추정하기 위한 split 과정
train_batch_size = n_class * train_size_for_each_label
test_batch_size = n_class * test_size_for_each_label

train_token_ls_split, train_tag_ls_split = nlp.extract_a_equally_splited_batch(train_df['Token'], train_df['Section'], train_batch_size)
test_token_ls_split, test_tag_ls_split =  nlp.extract_a_equally_splited_batch(test_df['Token'],test_df['Section'], test_batch_size)

<br>


<br>


In [9]:
' '.join(train_token_ls_split[0])

'아마 단독콘서트 서울 아트 마켓 쇼케이스 연극 소설가 구보 사람 서울 청계천로 한류 복합 창작 전문 공연장 CKL 스테이지 무대 에서 10월 동안 국악 무용 연극 가을 감성 자극 로운 문화 공연 펼쳐진다 27일 한국 콘텐츠 진흥 따르면 소리 연희 전공 여성 모인 국악 창작 그룹 아마 10월 3일 단독 콘서트 발아 Budding 선보인다 이번 무대 에는 형식 얽매 이지 않은 음악 어법 으로 시대 살아가는 여성 시각 이면 이야기 담는다세 상과 사람 대한 시선 나타낸 아스팔트 피는 미세먼지 에서 출발 고통 주는 로부터 벗어나 고픈 마음 노래 dusty city 우리 살아가는 시대 표현 혐오 시대 들려줄 예정 이다 아마 목소리 만으로 강렬한 메시지 전달 강한 여운 남긴다 10월 712일 열리는 국내 최대 규모 국제 예술 공연 마켓 2018 아트 마켓 우수 작품 해외 작품 선정 CKL 스테이지 에서 쇼케이스 펼친다 10일 없는 현실 살아가는 어둠 몸짓 주목 전과 셰익스피어 오셀로 탈춤 접목 오셀로 이아 고가 무대 오른다 11일 동시 살아가는 각자 다룬 동행 스웨덴 에서 시르커스 알폰 하이테크 뮤지컬 AM Somebody 예정 있다 12일 2018 한국 평론 협회 평론가 연기상 수상작 Voice of Acts 다양한 변주 보여주는 80 노파 저승 여행 이야기 차례 만난다 한국 문학사 발자국 남긴 소설가 구보 박태원 바탕 으로 옴니버스 연극 소설가 구보 사람 18일 부터 열흘 무대 오른다 2007년 예술의전당 자유 젊은 연극 선정 으로 일으킨 작품 으로 12 언어 연극 스튜디오 창립 12 주년 기념 창작 11년 만에 재연 극작가 이자 연출가 성기웅 지난 10 선보이는 구보 연작 작품 으로 1930년 경성 생활 풍속 다채롭 담아 냈다 CKL 스테이지 박태원 생가 약국 자리 구보 과거 현재 만나는 장소 로서 남다른 의미 있다 콘텐츠 진흥 운영 하는 CKL 스테이지 서울 중구 CKL 기업 지원 센터 지하 있는 한류 복합 창작 전문 공연장 이다 자세한 사항 CKL 스테이지 홈페

In [10]:
train_tag_ls_split[0]

'문화'

# Doc2Vec Parameter 튜닝

## **set ALPHA as default**

In [ ]:
%%time

result_dict = {'corpus_count' : [],
               'min_count' : [],
               'vector_size' : [],
               'window' : [],
               'n_epochs' : [],
               'accuracy' : [],
              }

testing_section_ls = ['경제','기업','사회','국제','부동산','증권','정치','IT과학','문화']

# 하이퍼 파라미터 튜닝 작업 수행

for size in [30000, len(train_df)]:
    x_split, y_split = nlp.extract_a_equally_splited_batch(train_df['Token'].tolist(), train_df['Section'].tolist(), size)

    for min_count in [10, 30, 50]:
        for vector_size in [100, 300]:
            for window in [5, 10]:
                for n_epochs in [10,30]:

                    # Doc2Vec 모델 생성
                    nlp.make_Doc2Vec_model(min_count = min_count,
                                           sample = 1e-05,
                                           vector_size = vector_size,
                                           window = window,
                                           dm_mean = 0,
                                           dm_concat = 0)

                    nlp.build_and_train_Doc2Vec_model(x_split,
                                                      y_split,
                                                      n_epochs = n_epochs)


                    model_name = 'Doc2Vec_dm=%s&cc=%s&vs=%s&win=%s&neg=%s&min=%s&sample=%s&epochs=%s'%(\
                                                                                                         nlp.Doc2Vec_model.dm,
                                                                                                           nlp.Doc2Vec_model.corpus_count,
                                                                                                           nlp.Doc2Vec_model.vector_size,
                                                                                                           nlp.Doc2Vec_model.window,
                                                                                                           nlp.Doc2Vec_model.negative,
                                                                                                           nlp.Doc2Vec_model.min_count,
                                                                                                           nlp.Doc2Vec_model.sample,
                                                                                                           nlp.Doc2Vec_model.epochs)
                    # Doc2Vec 모델 저장
                    #nlp.Doc2Vec_model.save('Doc2Vec_model/'+model_name)


                    '''
                    
                    X =nlp.infer_vectors_with_Doc2Vec(train_token_for_display[0])

                    from sklearn.decomposition import PCA

                    pca = PCA(n_components=2)
                    X_pca = pca.fit_transform(X)
                    scatter_df = pd.DataFrame(X_pca,
                                              index = train_tag_for_display[0],
                                              columns = ['x','y'])

                    plt.figure(figsize = (15, 15))

                    for section in testing_section_ls:
                        temp_df = scatter_df[scatter_df.index == section]
                        plt.scatter(temp_df['x'].values, temp_df['y'].values, label = section, c = np.random.rand(3,))

                    plt.legend(loc = 'best')
                    plt.savefig('Doc2Vec_model/images/'+model_name)

                    '''
                    # 각 레이블별 1000개씩 학습, 

                    # LR
                    X_train = nlp.infer_vectors_with_Doc2Vec(train_token_ls_split)
                    y_train = train_tag_ls_split

                    X_test = nlp.infer_vectors_with_Doc2Vec(test_token_ls_split)
                    y_test = test_tag_ls_split


                    clf = LogisticRegression(solver = 'sag',
                                             multi_class = 'multinomial')


                    clf.fit(X_train, y_train)
                    y_pred = clf.predict(X_test)

                    result_dict['corpus_count'].append(nlp.Doc2Vec_model.corpus_count)
                    result_dict['min_count'].append(nlp.Doc2Vec_model.min_count)
                    result_dict['vector_size'].append(nlp.Doc2Vec_model.vector_size)
                    result_dict['window'].append(nlp.Doc2Vec_model.window)
                    result_dict['n_epochs'].append(nlp.Doc2Vec_model.epochs)
                    result_dict['accuracy'].append(accuracy_score(y_pred, y_test))
                    
                    print(accuracy_score(y_pred, y_test))
                    
                    
    pd.DataFrame(result_dict).to_csv('Parameter_tuning_result-dbow.csv', index = False)

In [12]:
nlp.Doc2Vec_model.docvecs.vectors_docs.shape

(9, 300)

In [16]:
nlp.Doc2Vec_model.similar_by_word('트럼프')

[('도널드', 0.863656222820282),
 ('대통령', 0.7049350738525391),
 ('백악관', 0.6218736171722412),
 ('행정부', 0.6034835577011108),
 ('백악', 0.5987303853034973),
 ('미국', 0.5439364314079285),
 ('김정은', 0.529384970664978),
 ('폼페이', 0.5000470876693726),
 ('폭스뉴스', 0.48696279525756836),
 ('정상회담', 0.4868667721748352)]

<br>
